In [1]:
#!pip install gdown

In [2]:
!pip install --upgrade --no-cache-dir gdown

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-4.4.0-py3-none-any.whl size=14774 sha256=94ebb0a7325515ab845104372fcafdb2aee39fde623a9a3be91386e15366fa6a
  Stored in directory: /tmp/pip-ephem-wheel-cache-xrnehqu6/wheels/fb/c3/0e/c4d8ff8bfcb0461afff199471449f642179b74968c15b7a69c
Successfully built gdown
  Attempting uninstall: gdown
    Found existing installation: gdown 4.2.2
    Uninstalling gdown-4.2.2:
      Successfully uninstalled gdown-4.2.2


In [3]:
import gdown
import pandas as pd
import numpy as np
import random

In [4]:
!gdown --id 1hrC11y_r0veFdZqkwH1MSUxSv_CwJtyC
!gdown --id 1LggfAKKnsJ1iCOWQI7XdVsSRE-R0CLsJ
!gdown --id 1WiW7K8F-E8afXoeqyMalkOlfYATRuS9V
!gdown --id 1ILq5E5i4P6kneSziQwXP_bDCc2B7A7RE
!gdown --id 1Co7iQWJhJdqNFTlChd-xT1b88VAzuX4f
!gdown --id 1XO_bjqQc6Tq6jQdfas5hq0t2jTyjBaz1
!gdown --id 14OAYTUIw9BoCCSTFIiYzgPq-1XyP0dhQ

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1hrC11y_r0veFdZqkwH1MSUxSv_CwJtyC
To: /content/credits.csv
100% 190M/190M [00:02<00:00, 74.2MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1LggfAKKnsJ1iCOWQI7XdVsSRE-R0CLsJ
To: /content/keywords.csv
100% 6.23M/6.23M [00:00<00:00, 60.3MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: http

**movies_metadata.csv:** The main Movies Metadata file. Contains information on 45,000 movies featured in the Full MovieLens dataset. Features include posters, backdrops, budget, revenue, release dates, languages, production countries and companies.

**keywords.csv:** Contains the movie plot keywords for our MovieLens movies. Available in the form of a stringified JSON Object.

**credits.csv:** Consists of Cast and Crew Information for all our movies. Available in the form of a stringified JSON Object.

**links.csv:** The file that contains the TMDB and IMDB IDs of all the movies featured in the Full MovieLens dataset.

**links_small.csv:** Contains the TMDB and IMDB IDs of a small subset of 9,000 movies of the Full Dataset.

**ratings.csv:** Ratings of the Full Dataset.

**ratings_small.csv:** The subset of 100,000 ratings from 700 users on 9,000 movies.

In [6]:
movies_metadata = pd.read_csv('movies_metadata.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [20]:
movies_metadata.head(1)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


**1 Получим случайную матрицу эмбедингов для каждого фильма**

In [17]:
matrix_emb = pd.concat([movies_metadata[['id', 'original_title']], pd.DataFrame(np.random.random((movies_metadata.shape[0], 30)))], axis=1)

In [152]:
matrix_emb = matrix_emb.drop(matrix_emb[matrix_emb.id == '1997-08-20'].index)

In [148]:
matrix_emb = matrix_emb.drop(matrix_emb[matrix_emb.id == '2012-09-29'].index)

In [150]:
matrix_emb = matrix_emb.drop(matrix_emb[matrix_emb.id == '2014-01-01'].index)

In [156]:
matrix_emb['id'] = matrix_emb['id'].astype(int)

**2 Теперь напишем функцию train_test_split по юзеру по матрице рейтингов (юзер х фильмы = рейтинг)**

**Подаем на вход функции юзера и получаем два списка фильмов: на тренировочном будем учить, а фильмы из тестового набора искать в предсказаниях**


In [21]:
ratings = pd.read_csv('ratings.csv')

In [103]:
ratings.sample(5)

,userId,movieId,rating,timestamp
17404087,180619,155288,4.0,1469364037
24196202,251277,780,4.0,852832083
15203261,158219,7150,0.5,1088659728
11950944,123899,116897,4.0,1466140104
15603349,162417,2396,5.0,985103167


In [179]:
def train_test_split_films(dataframe, user):

  '''
     на вход подается название датафрейма с рейтингами
     и айди нужного пользователя

     эта функция выведет данные именно для одного пользователя
     можно доработать для большего количества, добавив цикл по
     передаваемому спику айдишников

     на выхлде получаем словарь, где
     ключ - id зрителя
     список 1 - айдишники train фильмов
     список 2 - оценки train фильмов
     список 3 - айдишники test фильмов
     список 4 - оценки test фильмов
  '''

  user_tt_data_dict = {}
  user_tt_data_list = []

  # выбираю данные по нужному юзеру и перемешиваю
  temp_df = dataframe[dataframe['userId'] == user].sample(dataframe[dataframe['userId'] == user].shape[0])[['movieId', 'rating']]

  # разбиваю их на трейн и тест 0.7 к 0.3
  train = temp_df.iloc[0:int(temp_df.shape[0]*0.7)]
  test = temp_df.iloc[int(temp_df.shape[0]*0.7):]
  
  # разделяю каждый набор на вектора фильмов и рейтингов и записываю в общий список
  user_tt_data_list.append(train['movieId'].tolist())
  user_tt_data_list.append(train['rating'].tolist())
  user_tt_data_list.append(test['movieId'].tolist())
  user_tt_data_list.append(test['rating'].tolist())

  # список со всеми векторами записываю в словарь под ключ - айди зрителя
  user_tt_data_dict[user] = user_tt_data_list

  return user_tt_data_dict

In [180]:
splitted_data = train_test_split_films(ratings, 270896)

**3 Для каждого юзера по train-фильмам находим средний вектор эмбединг через mean**

**Надо дополнительно домножить эмбединг-вектор на оценку фильма**

In [242]:
def get_avg_weightened_vec(dict_name, user_id, movies_emb_df):

  '''
     на вход подается название словаря с данными после сплита,
     айдишник нужного зрителя и название эмбединг-датасета с фильмами

     далее мы находим все эмбединг-вектора для train фильмов, домножаем
     каждый на оценку от зрителя и находим средний вектор
  '''

  # находим все вектора фильмов из train-выборки 
  temp_df = movies_emb_df.loc[movies_emb_df['id'].isin(dict_name[user_id][0])].merge(pd.DataFrame(dict_name[user_id][0], dict_name[user_id][1], columns=['id']).reset_index().rename(columns={'index':'rating'}), on='id')

  # домножаем эмбединг-вектора на веса-оценки и вычисляем средний вектор
  avg_weightened_vec = list(temp_df.drop(['id', 'original_title'], axis=1).mul(temp_df['rating'], axis=0).drop(['rating'], axis=1).mean())

  return avg_weightened_vec


In [246]:
res_vec = get_avg_weightened_vec(splitted_data, 270896, matrix_emb)

In [247]:
res_vec

[2.0006146378647722,
 2.0761992005766925,
 1.8500482216045584,
 1.702884255898778,
 1.921912465293808,
 2.1206757532985354,
 2.023732511886943,
 1.823972378592632,
 1.8726064614886804,
 2.092124690968725,
 2.139106718711734,
 2.115520425175656,
 2.0175882308001296,
 1.9268781131469397,
 1.882923552463653,
 2.2353942679225534,
 1.8757815477039022,
 2.059470693395709,
 1.9969831470348656,
 2.0924237435882134,
 1.8619188147984458,
 2.0517491780394157,
 2.1491330816696865,
 2.0667138058075256,
 1.9848267543934988,
 2.0776754112510982,
 1.9622717273911596,
 1.994578244630689,
 2.037494704248514,
 1.9202317702235305]

**4 Находим топ-близких фильмов, которые пользователь еще не смотрел (косинусное расстояние между средним вектором пользователя и всеми фильмами)**